In [5]:
%pip install openai pandas scikit-learn


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
import pandas as pd
import openai
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
import re
openai.api_key = "your_openai_api_key"



In [7]:
def load_so_data(path):
    questions = pd.read_csv(path)
    return pd.DataFrame(questions)

# Function to categorize by popularity
def categorize_popularity(df):
    total_questions = len(df)
    df_sorted = df.sort_values(by='ViewCount', ascending=False)
    highly_popular = df_sorted.iloc[:int(total_questions * 0.1)]
    unpopular = df_sorted.iloc[int(total_questions * 0.9):]
    average_popular = df_sorted.iloc[int(total_questions * 0.1):int(total_questions * 0.9)]
    df_sorted['Popularity'] = ''
    df_sorted.loc[highly_popular.index, 'Popularity'] = 'Highly Popular'
    df_sorted.loc[average_popular.index, 'Popularity'] = 'Average Popular'
    df_sorted.loc[unpopular.index, 'Popularity'] = 'Unpopular'
    return df_sorted

# Function to categorize by posting time
def categorize_posting_time(df):
    cutoff_date = '2022-11-30'
    df['CreationDate'] = pd.to_datetime(df['CreationDate'])
    df['Recency'] = df['CreationDate'].apply(lambda x: 'New' if x >= pd.to_datetime(cutoff_date) else 'Old')
    return df

# Function to apply SVM classifier
def classify_question_types(df, classifier, vectorizer):
    # load save model and add type so df dataframe
    return df

def sample_questions(df, number = 517):
    sampled_df = df.sample(n=number)
    return sampled_df

def store_data(df, csv_path):
    df.to_csv(csv_path, index=False)

In [ ]:
df = load_so_data('')
    
df = categorize_popularity(df)

df = categorize_posting_time(df)

# Placeholder for SVM classifier and vectorizer
classifier = SVC()
vectorizer = TfidfVectorizer()

# Classify question types
df = classify_question_types(df, classifier, vectorizer)

# Sample for manual analysis
manual_analysis_sample = sample_questions(df)

# Sample for linguistic analysis
linguistic_sample = sample_questions(df, 2000)

store_data(manual_analysis_sample, 'manual_analysis.csv')
store_data(linguistic_sample, 'linguistic_analysis.csv')

In [9]:
def generate_prompt(title, body, tags):
    prompt = f"""
    Question Title: {title}
    Question Body: {body}
    Tags: {tags}
    """
    return prompt

def get_chatgpt_response(prompt,image_url, model="gpt-4-vision"):
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {
                    "type": "image_url",
                    "image_url": {
                        "url": image_url,
                    },
                    },
                ],
                }
            ]
            )
        return response['choices'][0]['message']['content']
    except Exception as e:
        print(f"Error: {e}")
        return None

In [ ]:
output = []

for index, row in manual_analysis_sample.iterrows():
    try:
        title = row['question_title']
        body = row['question_body']
        tags = row['tags']
        image_url = row['image_url']
        human_answer = row['answer']

        prompt = generate_prompt(title, body, tags)

        gpt_answer = get_chatgpt_response(prompt, image_url)

        output.append({
            'question_title': title,
            'question_body': body,
            'tags': tags,
            'image_url': image_url,
            'human_answer': human_answer,
            'gpt_answer': gpt_answer
        })

    except Exception as e:
        print(f"Error processing row {index}: {e}")
        continue

output_df = pd.DataFrame(output)
output_df.to_csv("manual_analysis_with_gpt_answers.csv", index=False)

In [ ]:
output = []

for index, row in linguistic_sample.iterrows():
    try:
        title = row['question_title']
        body = row['question_body']
        tags = row['tags']
        image_url = row['image_url']
        human_answer = row['answer']

        prompt = generate_prompt(title, body, tags)

        gpt_answer = get_chatgpt_response(prompt, image_url)

        output.append({
            'question_title': title,
            'question_body': body,
            'tags': tags,
            'image_url': image_url,
            'human_answer': human_answer,
            'gpt_answer': gpt_answer
        })

    except Exception as e:
        print(f"Error processing row {index}: {e}")
        continue

output_df = pd.DataFrame(output)
output_df.to_csv("linguistic_sample_with_gpt_answers.csv", index=False)